In [386]:
# imports
import pandas as pd
import numpy as np
import xlrd as xl
import matplotlib.pyplot as plt
import os
import glob

In [387]:
#initialize main dataframe 
df = pd.DataFrame(columns=['Filename', 'Holder', 'Well', 'Signal', 'RT [min]', 'Type', 'Width [min]', 'Area', 'Height', 'Peak Symmetry'])
df

,Filename,Holder,Well,Signal,RT [min],Type,Width [min],Area,Height,Peak Symmetry


In [388]:
#iterates though a directory to find all possible excel files. path can be changed. 
#can change to an input if needed
path = r"C:\Users\aniko\repo\auto-lab\data\2022-03-10_t44_L1-15_triplicate.01 2022-03-10 09-34-30"

files_list = []
for root, directories, files in os.walk(path):
	for name in files:
		suffix = os.path.splitext(name)[-1]
		if(suffix == ".XLS"):
			files_list.append(os.path.join(root, name))

print(files_list[0])

C:\Users\aniko\repo\auto-lab\data\2022-03-10_t44_L1-15_triplicate.01 2022-03-10 09-34-30\001-P1-A1-A1.D\IRReport_2022-03-10_t44_L1-15_triplicate.0110 Mar 22   9_36 am -0600.XLS


In [389]:
def find_file(tester):
       datafile = ""
       holder = ""
       well = ""
       #using exceptions to break out of a double for loop :3
       class FileFound(Exception): pass
       try:
              for r in range (tester.shape[0]):
                     for c in range (tester.shape[1]):
                            if(tester.iat[r,c] == "Data file:"):
                                   raise FileFound
       except FileFound:
              datafile = tester.iat[r, c + 6]

       return datafile

In [390]:
def find_details(filename):
    fn_list = filename.split("\\")
    detail = fn_list[-1]
    detail_list = detail.split("-")
    return detail_list

In [391]:

def sig_iterator(row, col, filename, holder, well, tester):
    sig_name = tester.iat[row, col + 2]
    datarow = []
    
    #loops until end of signal information
    class SingleSig(Exception): pass

    try:
        for r in range (row + 2, tester.shape[0]):
            datarow = [filename, holder, well, sig_name]
            for c in range (col, tester.shape[1]):
                if(tester.iat[r,c] == "Sum"):
                    raise SingleSig
                if(pd.notnull(tester.iat[r,c])):
                    datarow.append(tester.iat[r,c])

            #append row to the dataframe        
            df.loc[len(df)] = datarow

    except SingleSig:
        ...

In [392]:
#main reader method: xls reader
#may need input as xls file
def xls_reader(tester):
    #finds filename and details:
    filename = find_file(tester)
    details = find_details(filename)
    holder = details[1]
    well = details[2]

    #checks for signals    
    for r in range (tester.shape[0]):
        for c in range (tester.shape[1]):
            if(tester.iat[r, c] == "Signal:"):
                sig_iterator(r, c, filename, holder, well, tester)

In [393]:
#iterator that houses main reader method
def lab_cleaner():
    for l in range (len(files_list)):
        #create new instance of 'tester'
        tester = pd.read_excel(files_list[l])
        #read tester
        xls_reader(tester)


lab_cleaner()

In [394]:
def to_xls(df):
    excelname = "Summary_" + path.split("\\")[-1] + ".xlsx"
    #write to xls file with 'excelname'
    with pd.ExcelWriter(excelname) as writer:  
        df.to_excel(writer)

to_xls(df)
